# Постановка задачи и выяснение общей картины

1. Разделите данные на две группы: органический трафик (прямые ссылки на сайт и поисковые переходы, за которые рекламодатель не платит); и рекламный трафик (всё остальное).

2. Сравните доли пользователей на каждом этапе воронки между двумя группами. Насколько статистически значимы отличия между долями? Примените знакомые вам статистические критерии (подсказка: можно рассмотреть критерий долей).

2. Посчитайте воронки для каждого канала коммуникации и выявите лучший и худший каналы по данным на этапах consideration, intent, purchase.

3. Клиент, представляющий онлайн-кинотеатр, хотел бы исключить хуже всего работающий канал. Аргументированно опишите, какой канал будет отключён и как можно перераспределить показы, чтобы увеличить продажи. Формулируйте гипотезы, полагаясь на результаты анализа, полученные в предыдущих блоках.

In [1]:
import pandas as pd
df_new = pd.read_csv('table_summary.csv')

In [3]:
def print_fullycomplete_and_percent(df):
    print('количество полностью заполненных объектов ',len(df.dropna()))
    print('процент полностью заполненных объектов из всей выборки ',round(len(df.dropna()) / len(df) * 100, 2 ))

In [5]:
print_fullycomplete_and_percent(df_new2)

количество полностью заполненных объектов  4877
процент полностью заполненных объектов из всей выборки  0.13


In [10]:
# есть пропуски в данных

In [4]:
# копируем новый датасет
df_new2 = df_new.copy() 

In [6]:
# итого получаем, что пустых данных нет
missing_values_percent = ( (df_new2.isna().sum() / len(df_new2))* 100 ).sort_values(ascending=False)
missing_values_percent 

profit          99.873183
user_id          0.000000
funnel_stage     0.000000
timestamp        0.000000
os               0.000000
ad_channel       0.000000
dtype: float64

In [7]:
df_new2 = df_new2.drop('profit', axis = 1)

In [8]:
print_fullycomplete_and_percent(df_new2)

количество полностью заполненных объектов  3845707
процент полностью заполненных объектов из всей выборки  100.0


In [9]:
# теперь нет пропусков в данных

Собираем в список все целевые действия и создаем новый признак равный целевому действию и не целевому действию

In [14]:
utm_medium_type_organic_not_organic = [
    'search', 
    'organic_traffic', 
    'native']
df_new2['utm_medium_type_organic_not_organic'] = df_new2['ad_channel'].apply(lambda x : 'organic' if x in utm_medium_type_organic_not_organic else 'not_organic')

Строим сводную таблицу с группировкой по типу траффика (органика / неорганика) и типу конверсии - в целефвое действие и нецелевое действие

In [20]:
df_new2.groupby(['utm_medium_type_organic_not_organic','funnel_stage'])['user_id'].count()


utm_medium_type_organic_not_organic  funnel_stage 
not_organic                          consideration     139612
                                     intent             74835
                                     interest         1489688
                                     purchase            1208
organic                              consideration     200494
                                     intent            206076
                                     interest         1730125
                                     purchase            3669
Name: user_id, dtype: int64

Нулевая гипотеза : кол-во конверсий в продажу среди органического трафика такое же, как и при не-органическом
Альтенативная гипотеза: кол-во конверсий выше при не-органическом траффике
Используем биномиальный критерий проверки гипотезы.

Создаем функцию статистического Биноминального критерия значимости

In [21]:
def p_bin(m1,n1,m2,n2,level_conf=0.05): 
    # на вход ф-ии подается кол-во успехов варианта А и общее число вариантов m1,n1 
    #и число успехов варанта Б и общее число вариантов m2, n2, 
    #а также уровень значимости - опционально (0.05 по умолчанию)
    from scipy import stats
    T = (m1/n1 - m2/n2)/((m1+m2)/(n1+n2)*(1 - (m1+m2)/(n1+n2))*(1/n1+1/n2))**0.5 #формула биномиального критерия вер-сти
    P = 1 - stats.norm.cdf(T)  # вычисляем p-значение
    # критичная область у нас со знаком БОЛЬШЕ, потому что задачался вопрос в альтернативной что ВАРИАНТ А был более значимым
    print(f"Stats {T}, p-value {P}")
    if (P < level_conf):
        print(f"На уровне значимости {level_conf} нулевая гипотеза отвергается в пользу альтернативной: Вариаент А более значим (значимая разница Варианта А перед вариантом Б есть)")
    else:
        print(f"На уровне значимости {level_conf} нулевая гипотеза принимается (нет значимой разницы между вариантом А и Б)")
#p_bin(46,1100,30,1000,0.1) Пример ф-иии и ответа:
# На уровне значимости 0.1 нулевая гипотеза отвергается в пользу альтернативной: Вариаент А более значим (значимая разница Варианта А перед вариантом Б есть)

In [22]:
p_bin(1208, 1208+1489688+74835+139612, 3669, 3669+1730125+206076+200494)

Stats -27.534331445524977, p-value 1.0
На уровне значимости 0.05 нулевая гипотеза принимается (нет значимой разницы между вариантом А и Б)


In [ ]:
# выясняем лучший и худший каналы по данным на этапах consideration, intent, purchase

In [51]:
ad_channel_list = [
'search',
'video' ,          
'banner'  ,            
'organic_traffic'  ,   
'partner_network',     
'native'   ,           
'rich'  ,               
'other' ]
for _ in ad_channel_list:
    _v_interest_consiferation = 100 * (df_new2.groupby(['ad_channel','funnel_stage'])['user_id'].count()[_][0]) / (df_new2.groupby(['ad_channel','funnel_stage'])['user_id'].count()[_][2])
    _v_conideartion_intent = 100 * (df_new2.groupby(['ad_channel','funnel_stage'])['user_id'].count()[_][1]) / (df_new2.groupby(['ad_channel','funnel_stage'])['user_id'].count()[_][0])
    _v_intent_purchase = 100 * (df_new2.groupby(['ad_channel','funnel_stage'])['user_id'].count()[_][3]) / (df_new2.groupby(['ad_channel','funnel_stage'])['user_id'].count()[_][1])
    print(_)
    print(_, "interest to consideration",  f"{_v_interest_consiferation:.2f}" )
    print(_, "consideration to intent ",  f"{_v_conideartion_intent:.2f}" )
    print(_, "intent to purcahse",  f"{_v_intent_purchase:.2f}" )
    print()
    
        

search
search interest to consideration 11.23
search consideration to intent  117.38
search intent to purcahse 1.96

video
video interest to consideration 10.26
video consideration to intent  0.95
video intent to purcahse 0.86

banner
banner interest to consideration 9.74
banner consideration to intent  84.12
banner intent to purcahse 1.72

organic_traffic
organic_traffic interest to consideration 12.43
organic_traffic consideration to intent  86.67
organic_traffic intent to purcahse 1.17

partner_network
partner_network interest to consideration 7.38
partner_network consideration to intent  110.93
partner_network intent to purcahse 1.64

native
native interest to consideration 12.24
native consideration to intent  15.78
native intent to purcahse 1.86

rich
rich interest to consideration 8.55
rich consideration to intent  106.70
rich intent to purcahse 0.44

other
other interest to consideration 9.61
other consideration to intent  103.31
other intent to purcahse 2.30



лучший и худший каналы

1. interest to consideration: лучший organic_traffic (12.43%) худший partner_network  (7.38%)
2. consideration to intent: лучший search (117.38%) худший video (0.95%)
3. intent to purcahse: лучший other (2.30%) худший video (0.86%)

## Исходя из данных анализа: ОТВЕТЫ

Значимой разницы между конверсий в продажу среди органического трафика и не-органического нет

более того, лучшая воронка продаж - на этапах это organic_traffic, search, наихудшие это video и rich

В количестве продаж лидирует 
banner (c хорошей воронкой banner interest to consideration 9.74 banner consideration to intent  84.12, banner intent to purcahse 1.72
organic_traffic (organic_traffic interest to consideration 12.43 organic_traffic consideration to intent  86.67 organic_traffic intent to purcahse 1.17) 
partner_network (partner_network interest to consideration 7.38 partner_network consideration to intent  110.9 partner_network intent to purcahse 1.64)

Следует отказаться от video, rich и переориентироваться на organic_traffic, search, partner_network, banner

In [52]:
#ниже группировка по каналам продаж и этапами воронки

In [37]:
df_new2.groupby(['ad_channel','funnel_stage'])['user_id'].count()

ad_channel       funnel_stage 
banner           consideration      47541
                 intent             39992
                 interest          487948
                 purchase             688
native           consideration      12597
                 intent              1988
                 interest          102953
                 purchase              37
organic_traffic  consideration      53592
                 intent             46447
                 interest          431003
                 purchase             542
other            consideration       1813
                 intent              1873
                 interest           18859
                 purchase              43
partner_network  consideration      24721
                 intent             27422
                 interest          334837
                 purchase             450
rich             consideration       4657
                 intent              4969
                 interest           54440
   